# Equação da onda elástica de 1ª ordem - 2 Camadas Líquido-sólido

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.axes_grid1 import make_axes_locatable
try:
    from examples.seismic.stiffness.model import ElasticModel
except:
    from examples.seismic.stiffness.model import ISOSeismicModel as ElasticModel

from examples.seismic import plot_velocity, TimeAxis, setup_geometry
from examples.seismic.stiffness.utils import C_Matrix, D, S, vec
from examples.seismic import RickerSource, AcquisitionGeometry, Receiver
from devito import *
from plot_functions import plot_animated_wavefield, plot_shotrecord, plot_wavefield_snaps

%matplotlib inline

## Parâmetros dos operadores de diferenças finitas

In [ ]:
space_order=16
time_order=1

## Dimensões do modelo

O modelo possui dimensões $w \times h = 2000\ \mathrm{m} \times 2000\ \mathrm{m}$, e um espaçamento de $\Delta x = \Delta z = 10\ \mathrm{m}$. Logo, a malha deverá ter 
$$
n_x = \frac{w}{\Delta x} + 1 = \frac{2000\ \mathrm{m}}{10\ \mathrm{m}} + 1 = 201
$$
pontos ao longo do eixo x, e
$$
n_z = \frac{h}{\Delta z} + 1 = \frac{2000\ \mathrm{m}}{10\ \mathrm{m}} + 1 = 201
$$
pontos ao longo do eixo z.

In [ ]:
shape = (201, 201)
spacing = (10., 10.)
origin = (0., 0.)
nbl=201

## Parâmetros físicos

<!-- O modelo possui 2 camadas cujas velocidades de propagação são

Camada | $v_P$ ($\mathrm{km/s}$) | $v_S$ ($\mathrm{km/s}$)
-|-|-
1|1.5|0.75
2|3.5|1.75 -->

In [ ]:
vp1 = 1.5

vp = vp1 * np.ones(shape, dtype=np.float32)

CASO_HOMOGENEO = True

# Remova o comentário das linhas abaixo para gerar um modelo de duas camadas
# vp2 = 3.5
# vp[:, shape[1]//2:] = vp2
# CASO_HOMOGENEO = False


vs = vp/2
rho = 1

In [ ]:
model = ElasticModel(vp=vp, vs=vs, rho=rho, origin=origin, shape=shape, spacing=spacing,
              space_order=space_order, nbl=nbl, bcs="damp")

In [ ]:
plot_velocity(model)

## Tempo de aquisição

O período de aquisição inicia em $t_0 = 0$ $\mathrm{ms}$ e termina em $t_n = 2500$ $\mathrm{ms}$. O comprimento do passo temporal, $\Delta t$, foi escolhido como o $\Delta t_{\text{crítico}}$ que, embora tenha sido calculado por outro método, satisfaz a condição de estabilidade:
$$
\Delta t \le \frac{1}{\max(v_P)\sqrt{\dfrac{1}{\Delta x^2} + \dfrac{1}{\Delta z^2}}}
$$

In [ ]:
t0 = 0.
tn = 1500.
dt = model.critical_dt

## Geometria

Como o valor mínimo do campo de velocidade é $1.5\ \mathrm{km/s}$, o espaçamento do modelo é $\Delta x = \Delta z = 10 \mathrm{m}$ e a ordem do operador de diferenças finitas para as derivadas espaciais é igual a $16$ ($F = 2$), a frequência máxima $f_{\mathrm{max}}$ da onda propagada deve satisfazer
$$
\begin{aligned}
f_{\mathrm{max}} &\le \frac{\min{(v_P, v_S)}}{F\max{\Delta x, \Delta z}}\\
    &= \frac{1.5\ \mathrm{km/s}}{2 \times 10\ \mathrm{m}}\\
    &= 0.075\ \mathrm{kHz}
\end{aligned}
$$
A assinatura da fonte será a fonte de Ricker, que possui uma frequência de pico $f_{\mathrm{peak}} \approx \dfrac{1}{3} f_{\mathrm{max}}$. Com base nisto, foi escolhida uma frequência máxima de $0.03\ \mathrm{kHz}$, resultando numa frequência de pico igual a $0.01\ \mathrm{kHz}$.

In [ ]:
f0 = 0.01

In [ ]:
geometry = setup_geometry(model, tn, f0=f0)
time_range = geometry.time_axis

Será utilizada uma fonte de Ricker posicionada no canto superior esquerdo do modelo

In [ ]:
src = RickerSource(name='src', grid=model.grid, f0=f0,
                   npoint=1, time_range=time_range)

In [ ]:
src.coordinates.data[0, 0] = model.domain_size[0]/2
src.coordinates.data[0, 1] = model.domain_size[1]/2

In [ ]:
src.show()

Há um receptor para cada ponto da malha na superfície ($z = 0\ \mathrm{km}$). Logo, existem $n_{\mathrm{rec}} = 501$ receptores espaçados por $10$ $\mathrm{m}$. Os receptores leem os campos $\tau$ e $\mathbf{v}$.

In [ ]:
def create_receivers(name, model, geometry):
    time_range = geometry.time_axis
    shape = model.shape
    rec = Receiver(name=name, grid=model.grid, npoint=shape[0], time_range=time_range)
    rec.coordinates.data[:, 0] = np.linspace(0, model.domain_size[0], num=shape[0])
    rec.coordinates.data[:, 1] = 0  # Depth is 0m
    return rec

In [ ]:
rec_tauxx = create_receivers('recTauxx', model, geometry)
rec_tauzz = create_receivers('recTauzz', model, geometry)
rec_tauxz = create_receivers('recTauxz', model, geometry)

rec_vx = create_receivers('recVx', model, geometry)
rec_vz = create_receivers('recVz', model, geometry)

#### Posição da fonte

In [ ]:
plot_velocity(model, source=src.coordinates.data,
              receiver=rec_tauxx.coordinates.data[::4, :])

## Definição da equação da onda elástica de 1ª ordem

Aqui é definida no *Devito* o sistema de equações

$$
\begin{cases}
    \dfrac{\partial \mathbf{\tau}}{\partial t} - CD^T\mathbf{v} = \mathbf{S}\\
    \dfrac{\partial \mathbf{v}}{\partial t} - D\mathbf{\tau} = 0
\end{cases}
$$

que é a formulação de propagação da onda elástica de 1ª ordem com densidade constante, em que:

- $\mathbf{\tau}$ é o tensor de estresses;
- $\mathbf{v}$ é o campo de velocidade de partícula;
- $C$ é a matriz de rigidez:

    $$
    C = \begin{bmatrix}
        \rho v_P^2 & \rho (v_P^2 - 2v_S^2) & 0\\
        \rho (v_P^2 - 2v_S^2) & \rho v_P^2 & 0\\
        0 & 0 & \rho v_S^2\\
    \end{bmatrix}
    $$

- $D$ é a matriz de derivadas espaciais:

    $$
    D = \begin{bmatrix}
        \partial_x & 0 & \partial_z\\
        0 & \partial_z & \partial_x\\
    \end{bmatrix}
    $$
    
- $\mathbf{S}$ é o termo fonte;

In [ ]:
tau = vec(TensorTimeFunction(name='tau', grid=model.grid, time_order=time_order, space_order=space_order))
v = VectorTimeFunction(name='v', grid=model.grid, time_order=time_order, space_order=space_order)

Cria-se funções auxiliares para armazenar os campos de tensão e velocidade de partícula. São salvos 60 campos distribuidos ao longo do período de aquisição.

In [ ]:
nsnaps = 60
factor = round(geometry.nt/nsnaps)
time_subsampled = ConditionalDimension(name='tsub', parent=model.grid.time_dim,factor=factor)

tausave = vec(TensorTimeFunction(name='tausave', grid=model.grid, time_order=time_order, space_order=space_order, 
                                 save=nsnaps, time_dim=time_subsampled))
vsave = VectorTimeFunction(name='vsave', grid=model.grid, time_order=time_order, space_order=space_order, 
                                 save=nsnaps, time_dim=time_subsampled)


Definição das EDPs

In [ ]:
C = C_Matrix(model,'vp-vs-rho')
vp = model.vp
rho = model.rho

In [ ]:
EDP_tau = tau.dt - C* S(v.forward)
EDP_v = 1/rho * v.dt - D(tau)

Geração *stencils* para a expressão dos campos de ondas futuros com base nestas EDPs.

In [ ]:
# Expressão para o campo de ondas futuro, P(r, t + dt)
stencil_tau = Eq(tau.forward, solve(EDP_tau, tau.forward))
stencil_v = Eq(v.forward, solve(EDP_v, v.forward))

Com base na expressão acima para o campo $P(\mathbf{x}, t + \Delta t)$, observa-se que o termo fonte deverá ter um coeficiente igual a $\Delta t$. Logo, a fonte $S$ deve ser injetada no campo de ondas futuro como $\Delta t S(\mathbf{x}, t)$

In [ ]:
# Símbolo do espaçamento temporal
s = model.grid.stepping_dim.spacing

# Expressão do termo fonte a ser injetada no stencil criado acima
src_term_xx = src.inject(tau[0].forward, expr = s * src)
src_term_zz = src.inject(tau[1].forward, expr = s * src)
src_term = src_term_xx + src_term_zz

rec_term = rec_tauxx.interpolate(tau[0].forward)\
        + rec_tauzz.interpolate(tau[1].forward)\
        + rec_tauxz.interpolate(tau[2].forward)\
        + rec_vx.interpolate(v[0].forward)\
        + rec_vz.interpolate(v[1].forward)

### Criação do operador de diferenças finitas

In [ ]:
op = Operator([stencil_v, stencil_tau] + src_term + [Eq(tausave, tau), Eq(vsave, v)] + rec_term, subs=model.spacing_map)

In [ ]:
op(dt=model.critical_dt)

### Registro sísmico e propagação dos campos

In [ ]:
if CASO_HOMOGENEO:
    plot_animated_wavefield(vsave[0].data, rec_vx.data, geometry, src.coordinates.data[0], 
                            model, 'Propagação do campo $V_x$', scale_factor=1/500,
                            wave_scale_factor=1/5
                        )
else:
    intf_positions = [
        (
            [model.origin[0], model.domain_size[0]],
            [model.domain_size[1]/2, model.domain_size[1]/2]
        ),
    ]

    plot_animated_wavefield(vsave[0].data, rec_vx.data, geometry, src.coordinates.data[0], 
                            model, 'Propagação do campo $V_x$', scale_factor=1/500, intf_positions=intf_positions,
                            wave_scale_factor=1/5
                        )